In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import re
from collections import Counter

%run Functions/Data_Cleaning_Functions.ipynb

In [2]:
# Set the maximum number of displayed rows to 1000
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

# Data Import

In [3]:
scraped_data = pd.read_csv("Data//Scraped Data//scraped_data.csv", low_memory = False)

Take a look at the scraped Data

In [4]:
scraped_data.head()

,Barzahlungspreis,Karosserieform,Fahrzeugart,Antriebsart,Sitzplätze,Türen,Kilometerstand,Erstzulassung,HU,Letzte Inspektion,Fahrzeughalter,Leistung,Getriebe,Hubraum,Gänge,Zylinder,Kraftstoff,Kraftstoffverbrauch,Schadstoffklasse,Umweltplakette,Komfort,Extras,Außenfarbe,Farbe der Innenausstattung,Innenausstattung,Zulassungskosten,Fahrleistung p.a.,url,date,time,model,brand,Nichtraucherfahrzeug,Leergewicht,Unterhaltung/Media,Sicherheit,CO₂-Emissionen,Fahrzeugzustand,Letzter Zahnriemenwechsel,Lackierung,Scheckheftgepflegt,Angebotsnummer,CO₂-Klasse,Farbe laut Hersteller,Energieeffizienzklasse,CO₂-Effizienz,Schlüsselnummer,Garantie,Baujahr,Bruttodarlehensbetrag,Bearbeitungsgebühren,Sollzins geb. p.a.,Verfügbarkeit,Verfügbar ab,Andere Energieträger,Sonderzahlung,Taxi oder Mietwagen,Elektrische Reichweite7,Batteriebesitz,Stromverbrauch,Elektrische Reichweite,Ladezeit von 10% bis 80%,"€ 0,-",zul. Gesamtgewicht,zul. Zuggewicht,Laderaumhöhe,Laderaumbreite,Laderaumlänge,Achsen,Nutzlast,Radstand,Ladevolumen,Pkw Zulassung,Energieverbrauch,Anderer Energieträger,Energiekosten bei 15.000 km Jahresfahrleistung,Kraftstoffpreis,Kraftfahrzeugsteuer,Elektrische Reichweite (EAER),Kraftstoffverbrauch bei entladener Batterie,Elektrische Reichweite (EAER)7,Stromverbrauch bei rein elektrischem Antrieb,Mögliche CO₂-Kosten über die nächsten 10 Jahre (15.000 km/Jahr),Strompreis
0,"€ 19.450,-",Cabrio,Gebraucht,Heck,2.0,2.0,17.610 km,07/1992,05/2025,05/2023,2.0,37 kW (50 PS),Schaltgetriebe,1.600 cm³,4.0,4.0,Super 95,"8,0 l/100 km (komb.)",Euro 1,1 (Keine),Lederausstattung,Sommerreifen,Rot,Rot,Vollleder,Keine Angabe,10.000 km,https://www.autoscout24.de/angebote/mercedes-b...,2024-05-11,10-42-33,170,Mercedes-Benz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"€ 2.150,-",Kleinwagen,Gebraucht,Front,5.0,5.0,223.000 km,05/2001,07/2025,NaN,NaN,75 kW (102 PS),Schaltgetriebe,1.598 cm³,5.0,4.0,Super E10 95,NaN,NaN,4 (Grün),Elektr. FensterheberElektrische SeitenspiegelK...,E10-geeignetSommerreifen,Blau,Grau,Velours,Keine Angabe,10.000 km,https://www.autoscout24.de/angebote/mercedes-b...,2024-05-11,10-42-54,170,Mercedes-Benz,Ja,1.115 kg,CDRadio,BeifahrerairbagFahrerairbagZentralverriegelung...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"€ 1.500,-",Limousine,Gebraucht,Front,5.0,5.0,196.898 km,10/2003,10/2024,10/2022,2.0,70 kW (95 PS),Schaltgetriebe,1.698 cm³,5.0,4.0,Diesel,"4,9 l/100 km (komb.)6,3 l/100 km (innerorts)4,...",Euro 3,3 (Gelb),ArmlehneElektr. FensterheberElektrische Seiten...,AlufelgenStahlfelgenWinterreifen,Silber,Grau,Stoff,Keine Angabe,10.000 km,https://www.autoscout24.de/angebote/mercedes-b...,2024-05-11,10-43-00,170,Mercedes-Benz,Ja,NaN,Radio,ABSBeifahrerairbagESPFahrerairbagServolenkungZ...,131 g/km (komb.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"€ 4.000,-",Kleinwagen,Gebraucht,NaN,5.0,5.0,127.545 km,01/2005,02/2025,02/2023,2.0,85 kW (116 PS),Automatik,1.699 cm³,NaN,NaN,Super 95,NaN,NaN,NaN,Elektr. FensterheberKlimaanlagePanoramadachSch...,Allwetterreifen,Silber,Schwarz,Teilleder,Keine Angabe,10.000 km,https://www.autoscout24.de/angebote/mercedes-b...,2024-05-11,10-43-05,170,Mercedes-Benz,Ja,NaN,NaN,AbstandswarnerBeifahrerairbagFahrerairbagZentr...,NaN,Unfallfahrzeug,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"€ 2.000,-",Kleinwagen,Gebraucht,Front,5.0,5.0,176.334 km,07/2005,05/2024,01/2013,2.0,85 kW (116 PS),Schaltgetriebe,1.699 cm³,5.0,4.0,Super 95,"5,9 

# Entry Cleaning

Clean dataframe from observations where the values have shifted between the columns during scraping

In [5]:
print("########################## 2. Entry Cleaning ########################## ")
print("")
allowance_dict = {"Umweltplakette" : ["1 (Keine)", "4 (Grün)", "3 (Gelb)", "2 (Rot)", np.nan],
                 "Antriebsart" : ['Heck', 'Front', 'Allrad', np.nan],
                 "Scheckheftgepflegt" : ["Ja", "Nein", np.nan]}

df_step2 = clean_from_rows_with_shifted_columns(scraped_data)
print("")
print("########################## Finished 2. Entry Cleaning ##########################")
print("")
print("")
print("")
print("")

########################## 2. Entry Cleaning ########################## 

Dropped 4294 entries due to shifted columns!

########################## Finished 2. Entry Cleaning ##########################






# Column Cleaning

## Dropping Columns that contain "Unnamed"

##### Drop Columns which contain "Unnamed"

In [6]:
print("########################## 3. Column Cleaning ##########################")
print("")
print("#### 3.1 Dropping Columns that contain 'Unnamed' ####")
print("")
# drop unnamed columns
df_step3_1 = drop_unnamed_columns(df_step2)

print("")


########################## 3. Column Cleaning ##########################

#### 3.1 Dropping Columns that contain 'Unnamed' ####

Dropping 0 columns:
[]



## Dropping unwanted columns

Inspect unpopular columns

In [7]:
missing_values_info(df_step3_1)

,Missing Values,% of Total Values
"€ 0,-",558517,100.0
Pkw Zulassung,558516,100.0
Strompreis,558514,100.0
zul. Zuggewicht,558493,100.0
Ladevolumen,558491,100.0
Nutzlast,558468,100.0
Achsen,558456,100.0
Stromverbrauch bei rein elektrischem Antrieb,558453,100.0
Radstand,558451,100.0
Laderaumhöhe,558442,100.0


In [8]:
columns_to_drop = ["€ 0,-", "Pkw Zulassung", "Ladevolumen", "zul. Zuggewicht", "Achsen", "Nutzlast", "Radstand", "Laderaumhöhe",
                   "Laderaumbreite", "Laderaumlänge", "zul. Gesamtgewicht", "Sonderzahlung", "Verfügbarkeit", "Batteriebesitz",
                   "Letzter Zahnriemenwechsel", "Verfügbar ab", "Fahrzeugstand", "Energieeffizienzklasse", "CO₂-Effizienz",
                   "Bearbeitungsgebühren", "Bruttodarlehensbetrag", "Sollzins geb. p.a.", "Letzte Inspektion", "Baujahr",
                  "CO₂-Klasse", "Schlüsselnummer", "Gänge", "Angebotsnummer", "Farbe laut Hersteller",
                   "Farbe der Innenausstattung", "Andere Energieträger", "Fahrzeugzustand",
                   "Ladezeit von 10% bis 80%", "Fahrleistung p.a.", "Zulassungskosten", "HU"]

In [9]:
print("#### 3.2 Dropping Unwanted Columns ####")
print("")
df_step3_2 = drop_unwanted_columns(df_step3_1, columns_to_drop)
print("")

#### 3.2 Dropping Unwanted Columns ####

Dropped 36 columns:
['€ 0,-', 'Pkw Zulassung', 'Ladevolumen', 'zul. Zuggewicht', 'Achsen', 'Nutzlast', 'Radstand', 'Laderaumhöhe', 'Laderaumbreite', 'Laderaumlänge', 'zul. Gesamtgewicht', 'Sonderzahlung', 'Verfügbarkeit', 'Batteriebesitz', 'Letzter Zahnriemenwechsel', 'Verfügbar ab', 'Fahrzeugstand', 'Energieeffizienzklasse', 'CO₂-Effizienz', 'Bearbeitungsgebühren', 'Bruttodarlehensbetrag', 'Sollzins geb. p.a.', 'Letzte Inspektion', 'Baujahr', 'CO₂-Klasse', 'Schlüsselnummer', 'Gänge', 'Angebotsnummer', 'Farbe laut Hersteller', 'Farbe der Innenausstattung', 'Andere Energieträger', 'Fahrzeugzustand', 'Ladezeit von 10% bis 80%', 'Fahrleistung p.a.', 'Zulassungskosten', 'HU']



## Only Keeping Wanted Columns

In [10]:
wanted_columns = [
    'Barzahlungspreis',
    'brand',
    'model',
    'Kilometerstand',
    'Leistung',
    'Kraftstoff',
    'Antriebsart',
    'Karosserieform',
    'Fahrzeugart',
    'Sitzplätze',
    'Türen',
    'Fahrzeughalter',
    'Getriebe',
    'Hubraum',
    'Zylinder',
    'Kraftstoffverbrauch',
    'Schadstoffklasse',
    'Umweltplakette',
    'Komfort',
    'Extras',
    'Außenfarbe',
    'Innenausstattung',
    'url',
    'Nichtraucherfahrzeug',
    'Leergewicht',
    'Unterhaltung/Media',
    'Sicherheit',
    'CO₂-Emissionen',
    'Lackierung',
    'Scheckheftgepflegt',
    'Garantie',
    'Taxi oder Mietwagen',
    'Stromverbrauch',
    'Elektrische Reichweite',
    "Elektrische Reichweite7",
    'date',
    'Erstzulassung',
    'time']

In [11]:
df_step3_2 = df_step3_2[wanted_columns]

## Merge Columns

### Merge Elektrische Reichweite & Elektrische Reichweite7

In [12]:
df_step3_2

,Barzahlungspreis,brand,model,Kilometerstand,Leistung,Kraftstoff,Antriebsart,Karosserieform,Fahrzeugart,Sitzplätze,Türen,Fahrzeughalter,Getriebe,Hubraum,Zylinder,Kraftstoffverbrauch,Schadstoffklasse,Umweltplakette,Komfort,Extras,Außenfarbe,Innenausstattung,url,Nichtraucherfahrzeug,Leergewicht,Unterhaltung/Media,Sicherheit,CO₂-Emissionen,Lackierung,Scheckheftgepflegt,Garantie,Taxi oder Mietwagen,Stromverbrauch,Elektrische Reichweite,Elektrische Reichweite7,date,Erstzulassung,time
0,"€ 19.450,-",Mercedes-Benz,170,17.610 km,37 kW (50 PS),Super 95,Heck,Cabrio,Gebraucht,2.0,2.0,2.0,Schaltgetriebe,1.600 cm³,4.0,"8,0 l/100 km (komb.)",Euro 1,1 (Keine),Lederausstattung,Sommerreifen,Rot,Vollleder,https://www.autoscout24.de/angebote/mercedes-b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-11,07/1992,10-42-33
1,"€ 2.150,-",Mercedes-Benz,170,223.000 km,75 kW (102 PS),Super E10 95,Front,Kleinwagen,Gebraucht,5.0,5.0,NaN,Schaltgetriebe,1.598 cm³,4.0,NaN,NaN,4 (Grün),Elektr. FensterheberElektrische SeitenspiegelK...,E10-geeignetSommerreifen,Blau,Velours,https://www.autoscout24.de/angebote/mercedes-b...,Ja,1.115 kg,CDRadio,BeifahrerairbagFahrerairbagZentralverriegelung...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-11,05/2001,10-42-54
2,"€ 1.500,-",Mercedes-Benz,170,196.898 km,70 kW (95 PS),Diesel,Front,Limousine,Gebraucht,5.0,5.0,2.0,Schaltgetriebe,1.698 cm³,4.0,"4,9 l/100 km (komb.)6,3 l/100 km (innerorts)4,...",Euro 3,3 (Gelb),ArmlehneElektr. FensterheberElektrische Seiten...,AlufelgenStahlfelgenWinterreifen,Silber,Stoff,https://www.autoscout24.de/angebote/mercedes-b...,Ja,NaN,Radio,ABSBeifahrerairbagESPFahrerairbagServolenkungZ...,131 g/km (komb.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-11,10/2003,10-43-00
3,"€ 4.000,-",Mercedes-Benz,170,127.545 km,85 kW (116 PS),Super 95,NaN,Kleinwagen,Gebraucht,5.0,5.0,2.0,Automatik,1.699 cm³,NaN,NaN,NaN,NaN,Elektr. FensterheberKlimaanlagePanoramadachSch...,Allwetterreifen,Silber,Teilleder,https://www.autoscout24.de/angebote/mercedes-b...,Ja,NaN,NaN,AbstandswarnerBeifahrerairbagFahrerairbagZentr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-11,01/2005,10-43-05
4,"€ 2.000,-",Mercedes-Benz,170,176.334 km,85 kW (116 PS),Super 95,Front,Kleinwagen,Gebraucht,5.0,5.0,2.0,Schaltgetriebe,1.699 cm³,4.0,"5,9 l/100 km (komb.)",Euro 4,4 (Grün),Elektr. FensterheberElektrische SeitenspiegelK...,AllwetterreifenAlufelgenGepäckraumabtrennung,Silber,Teilleder,https://www.autoscout24.de/angebote/mercedes-b...,Ja,NaN,CDRadio,ABSFahrerairbagServolenkungWegfahrsperre,NaN,Metallic,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-11,07/2005,10-43-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562806,"€ 174.911,-",porsche,sonstige,133 km,353 kW (480 PS),Super Plus 98,Allrad,Coupé,Gebraucht,4.0,2,1.0,Automatik,2.981 cm³,6.0,"10,8 l/100 km (komb.)",Euro 6d,4 (Grün),2-Zonen-Klimaautomatik360° KameraArmlehneBerga...,AlufelgenElektronische ParkbremseInnenspiegel ...,Silber,Vollleder,https://www.autoscout24.de/angebote/porsche-ot...,Ja,1.670 kg,Android AutoApple CarPlayBluetoothBordcomputer...,ABSAbstandswarnerAlarmanlageBeifahrerairbagBi-...,246 g/km (komb.),Metallic,Ja,NaN,NaN,NaN,NaN,NaN,2024-05-12,06/2023,16-37-17
562807,"€ 124.850,-",porsche,sonstige,4.700 km,309 kW (420 PS),Benzin,Heck,Cabrio,Gebraucht,2.0,2,1.0,Automatik,3.995 cm³,6.0,"10,7 l/100 km (komb.)",Euro 6d,4 (Grün),2-Zonen-KlimaautomatikBeheizbares LenkradEinpa...,AlufelgenInnenspiegel automatisch abblendendPa...,Schwarz,Teilleder,https://www.autoscout24.de/angebote/porsche-ot...,NaN,1.525 kg,Apple CarPlayBordcomputerCDDAB-RadioRadioSound...,AlarmanlageBeifahrerairbagBlendfreies Fernlich...,242 g/km (komb.),Andere,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-12,04/2023,16-37-19
562808,"€ 97.850,-",porsche,sonstige,14.000 km,294 kW (400 PS),Benzin,Heck,Cabrio,Gebraucht,2.0,2,1.0,Automatik,3.995 cm³,6.0,"10,1 l/100 km (komb.)",Euro 6d,4 (Grün),2-Zonen-KlimaautomatikBeheizbares LenkradEinp

In [13]:
column1 = "Elektrische Reichweite"
column2 = "Elektrische Reichweite7"

df_step3_3_1 = merge_columns(df_step3_2, column1, column2)

In [14]:
print("########################## Finished 3. Column Cleaning ##########################")
print("")
print("")
print("")
print("")

########################## Finished 3. Column Cleaning ##########################






# Converting Strings to Floats/Integers

In [15]:
print("########################## 4. Convert Strings to Floats/Integers ##########################")

########################## 4. Convert Strings to Floats/Integers ##########################


## Convert Strings to other Strings

### Single Changes

Convert all "Nein" to "0" and all "Ja" to "12" for Garantie

In [16]:
column_name = "Garantie"
replacement_dict_garantie = {"Nein" : "0",
                             "Ja" : "12"}


df_step4_1_1 = replace_values_single(df_step3_3_1, column_name, replacement_dict_garantie)

In [17]:
column_name = "brand"
replacement_dict_garantie = {"porsche" : "Porsche",
                            "audi" : "Audi",
                            "opel" : "Opel",
                            "skoda" : "Skoda",
                            "toyota" : "Toyota"}


df_step4_1_1 = replace_values_single(df_step4_1_1, column_name, replacement_dict_garantie)

### Multiple Changes

Convert all "," to "." (and similar changes) for the following columns:
- Stromverbrauch
- Leistung
- Hubraum
- Leergewicht
- CO₂-Emissionen
- Kilometerstand

In [18]:
replacement_dict = {"Stromverbrauch" : {"," : "."},
                    "Leistung" : {"," : "."},
                    "Hubraum" : {"," : "."},
                   "Leergewicht" : {"," : "."},
                   "CO₂-Emissionen" : {"," : "."},
                   "Kilometerstand" : {"." : ""}}


df_step4_1_2 = replace_values_multiple(df_step4_1_1, replacement_dict)

### Remove leading and trailing whitespaces

In [19]:
column_names = ["brand", "model"]

df_step4_1_3 = remove_trailing_whitespace(df_step4_1_2, column_names)

### Replace Kraftstoff String with the most prominent Kraftstoff available in the initial Kraftstoff String

In [20]:
df_step4_1_4 = transform_fuel_types(df_step4_1_3, "Kraftstoff")

## Split Strings and turn to Floats

### Garantie, Stromverbrauch, Leistung, Hubraum & Leergewicht to Float

In [21]:
split_dict = {"Garantie" : " ",
             "Stromverbrauch" : " ",
             "Leistung" : " ",
             "Hubraum" : " ",
             "Leergewicht" : " "}

df_step4_2_1 = split_string_to_1integer(df_step4_1_4, split_dict)

### CO₂-Emissionen, Kilometerstand & Elektrische Reichweite to Float

In [22]:
split_dict = {"CO₂-Emissionen" : " ",
             "Kilometerstand" : " ",
             "Elektrische Reichweite" : " "}

df_step4_2_2 = split_string_to_1integer_complex(df_step4_2_1, split_dict)

## Convert Miscellaneous to Floats

### Converting Kraftstoffverbrauch to Floats

In [23]:
column_name = "Kraftstoffverbrauch"

df_step4_3_1 = convert_fuel_consumption(df_step4_2_2, column_name)

### Converting Barzahlungspreis to Floats

In [24]:
column_name = "Barzahlungspreis"

df_step4_3_2 = convert_barzahlungspreis_to_float(df_step4_3_1, column_name)

In [25]:
print("")
print("########################## Finished 4. Converting Strings to Floats/Integers ##########################")
print("")
print("")
print("")
print("")


########################## Finished 4. Converting Strings to Floats/Integers ##########################






# Missing Values

## Missing Values to Default Values

In [26]:
print("########################## 5. Missing Values ##########################")

########################## 5. Missing Values ##########################


Default value for:

- Nichtraucherfahrzeug
- Garantie
- Scheckheftgepflegt
- Taxi oder Mietwagen
- Elektische Reichweite
- Stromverbrauch

In [27]:
# set to which default values missing values in specified columns should be transformed
replacement_dict = {"Scheckheftgepflegt" : ["Nein"],
                    "Nichtraucherfahrzeug" : ["Nein"],
                    "Garantie" : [0],
                    "Taxi oder Mietwagen" : ["Nein"],
                    "Stromverbrauch" : [0],
                    "Elektrische Reichweite" : [0],
                    "Lackierung" : ["Keine Angabe"],
                    "Außenfarbe" : ["keine"]
                   }

df_step5_1 = missing_values_to_default(df_step4_3_2, replacement_dict)

## Missing values in specific columns to mean values 

### Mean Value for the Age of the car

Fill missing values with values from other observations of the same year:
- Erstzulassung

Get a mean value for the amount of Fahrzeughalter dependent on the mean of Fahrzeughalter from cars with the same year of "Erstzulassung".

In [28]:
column_to_mean = "Fahrzeughalter"
grouping_variable = "Erstzulassung"

df_step5_2_1 = get_median_value_based_on_erstzulassung(df_step5_1, column_to_mean, grouping_variable)

### Mean Values based on model and brand

Fill missing values with the most prominent values from other observations of the same model and brand for:

Grouping over model and brand for:

- Antriebsart
- Sitzplätze
- Türen

In [29]:
replacement_columns = ["Antriebsart", "Sitzplätze", "Türen"]

group_column1 = "brand"
group_column2 = "model"

df_step5_2_2 = fill_missing_with_mode_by_2groups(df_step5_2_1, replacement_columns,
                                                 group_column1, group_column2)

KeyboardInterrupt: 

### Mean Values based on model, brand and Leistung

Fill missing values with the most prominent values from other observations of the same model, brand and Leistung for:

Grouping over model, brand and Leistung for:

- Schadstoffklasse 
- Umweltplakette
- Getriebe
- Hubraum
- CO₂-Emissionen
- Zylinder
- Leergewicht
- Kraftstoff

In [ ]:
replacement_columns = ["Schadstoffklasse", "Umweltplakette", "Getriebe", "Hubraum",
                       "CO₂-Emissionen", "Zylinder", "Leergewicht", "Kraftstoff"]

group_column1 = "brand"
group_column2 = "model"
group_column3 = "Leistung"

df_step5_2_3 = fill_missing_with_mode_by_3groups(df_step5_2_2, replacement_columns, group_column1, group_column2, group_column3)

### Evaluation of Missing Values

In [ ]:
missing_values_info(df_step5_2_3)

In [ ]:
print("")
print("########################## Finished 5. Missing Values ##########################")
print("")
print("")
print("")
print("")

# Split Multi Feature Columns

In [ ]:
print("########################## 6. Split Multi Feature Columns ##########################")
print("")

Several Columns have many features in one column. These have to be split into lists or dummies:
- Komfort
- Innenausstattung
- Unterhaltung/Media
- Sicherheit
- Extras

In [ ]:
processing_type = "extra_columns" # extra_columns or count

## And turn to counts

Convert the Feature String into a Count indicating how many features the column represents

In [ ]:
if processing_type == "count":

    features_to_count = ["Sicherheit", "Innenausstattung", "Unterhaltung/Media", "Extras", "Komfort"]

    df_step6 = count_strings_in_column(df_step5_2_3, features_to_count)

Check results

In [ ]:
if processing_type == "count":

    print(df_step6[features_to_count][:2])

## And create extra Columns for them

Create a Dummy Column for each unique Feature from Sicherheit, Innenausstattung, Unterhaltung/Media, Extras and Komfort

In [ ]:
print("#### 6.2 Creating Extra Column for Multi-Feature Columns ####")
print("")
if processing_type == "extra_columns":
    
    features_to_columns = ["Sicherheit", "Innenausstattung", "Unterhaltung/Media", "Extras", "Komfort"]

    
    df_step6 = create_feature_columns(df = df_step5_2_3, column_names = features_to_columns)

print("")

Drop the old Columns

In [ ]:
print("Dropping the old Multi-Feature Columns")
print("")
if processing_type == "extra_columns":
    
    columns_to_drop = ["Sicherheit", "Innenausstattung", "Unterhaltung/Media", "Extras", "Komfort"]

    df_step6 = drop_unwanted_columns(df_step6, columns_to_drop)

print("")
print("########################## Finished 6. Split Multi Feature Columns ##########################")
print("")
print("")
print("")
print("")

Check Results

In [ ]:
df_step6[:2]

# Date Columns

In [ ]:
print("########################## 7. Date Columns ##########################")

## Scraped "date" and "time" to timestamp

### Create New Variable

In [ ]:
date_column = "date"
time_column = "time"

df_step7_1_1 = convert_to_timestamp(df_step6, date_column, time_column)

### Delete old date scraped columns

In [ ]:
print("Dropping Old Date Columns")
print("")
columns_to_drop = ["time", "date"]

df_step7_1_2 = drop_unwanted_columns(df_step7_1_1, columns_to_drop)
print("")

## Erstzulassung to Age

### Split Erstzulassung to "Erstzulassung_Jahr" and "Erstzulassung_Monat"

Split column from MM/YYYY into two columns containing the year and the month as a string

In [ ]:
df_step7_2_1 = split_month_year(df_step7_1_2, "Erstzulassung")

### Create Age Variable from that column (and delete Zulassungsmonat/Jahr)

In [ ]:
# set the registration date variables
registration_month_variable = "Erstzulassung_Monat"
registration_year_variable = "Erstzulassung_Jahr"

# run the transformation
df_step7_2_2 = convert_zulassung_to_age(df_step7_2_1, registration_year_variable, registration_month_variable)

### Drop the used columns

In [ ]:
print("Droppping Erstzulassung Column")
print("")
columns_to_drop = ["Erstzulassung", "Erstzulassung_Monat"]

df_step7_2_3 = drop_unwanted_columns(df_step7_2_2, columns_to_drop)

print("")
print("########################## Finished 7. Date Columns ##########################")
print("")
print("")
print("")
print("")

# Drop Observations that contain NaNs 

In [ ]:
print("########################## 8. Drop Observations that contain NaNs ##########################")
print("")

Check on missing values

In [ ]:
missing_values_info(df_step7_2_3)

## Drop all observations that contain NaNs

In [ ]:
# drop observations
df_step8 = df_step7_2_3.dropna()

In [ ]:
# get dropping info
length_before = len(df_step7_2_3)
length_after = len(df_step8)

# print dropping info
print(f"Dropped {length_before-length_after} observations due to incomplete values!")
print("")
print("########################## Finished 8. Drop Observations that contain NaNs ##########################")
print("")
print("")
print("")
print("")

# Outlier Handling

In [ ]:
print("########################## 9. Outlier Handling ##########################")
print("")

For each combination of Hersteller, Modell and Leistung, delete the observations with the lowest and highest 5% of Prices in this car category.

In [ ]:
group_cols = ["brand", "model", "Leistung"]
target_col = "Barzahlungspreis"

lower_quantile = 0.05
upper_quantile = 0.95

df_step9 = remove_outliers(df = df_step8,
                           group_cols = group_cols, target_col = target_col,
                           lower_quantile = lower_quantile, upper_quantile = upper_quantile)

In [ ]:
# inital length
len_init = len(df_step8)

# new length
len_new = len(df_step9)

# calculate how many observations have been deleted
n_deleted = len_init-len_new

# print info
print(f"Lower Bound: {lower_quantile}\nUpper Bound: {upper_quantile}\nObservations deleted during outlier handling: {n_deleted} [{(n_deleted/len_init)*100:.2f}%]")
print("")
print("########################## Finished 9. Outlier Handling ##########################")
print("")
print("")
print("")
print("")

# Rename Columns

In [ ]:
print("########################## 10. Rename Columns ##########################")

In [ ]:
rename_dict = {"Barzahlungspreis" : "Preis",
              "Leistung" : "Leistung_PS",
              "CO₂-Emissionen" : "CO2-Emissionen",
              "model" : "Modell",
              "brand" : "Hersteller",
              "url" : "URL"}

df_step10_1 = df_step9.rename(columns = rename_dict)

Translate Columns to English Language

In [ ]:
translation_dict = {
    'Preis': 'Price',
    'Hersteller': 'Manufacturer',
    'Modell': 'Model',
    'Kilometerstand': 'Mileage',
    'Leistung_PS': 'Power_HP',
    'Kraftstoff': 'Fuel',
    'Antriebsart': 'Drive_type',
    'Karosserieform': 'Body_type',
    'Fahrzeugart': 'Vehicle_type',
    'Sitzplätze': 'Seats',
    'Türen': 'Doors',
    'Fahrzeughalter': 'Owners',
    'Getriebe': 'Transmission',
    'Hubraum': 'Displacement',
    'Zylinder': 'Cylinders',
    'Kraftstoffverbrauch': 'Fuel_consumption',
    'Schadstoffklasse': 'Emission_class',
    'Umweltplakette': 'Environmental_sticker',
    'Komfort': 'Comfort',
    'Extras': 'Extras',
    'Außenfarbe': 'Exterior_color',
    'Innenausstattung': 'Interior_features',
    'URL': 'URL',
    'Nichtraucherfahrzeug': 'Non_smoker_vehicle',
    'Leergewicht': 'Curb_weight',
    'Unterhaltung/Media': 'Entertainment/Media',
    'Sicherheit': 'Safety',
    'CO2-Emissionen': 'CO2_emissions',
    'Lackierung': 'Paint',
    'Scheckheftgepflegt': 'Full_service_history',
    'Garantie': 'Warranty',
    'Taxi oder Mietwagen': 'Taxi_or_rental',
    'Stromverbrauch': 'Electricity_consumption',
    'Elektrische Reichweite': 'Electric_range',
    'date_scraped': 'Date_scraped',
    'Alter': 'Age'}

df_step10_2 = df_step10_1.rename(columns = translation_dict)

In [ ]:
print("")
print("########################## Finished 10. Renaming Columns ##########################")
print("")
print("")
print("")
print("")

# Rearrange Columns

In [ ]:
print("########################## 11. Rearrange Columns ##########################")

In [ ]:
first_columns = ["Price", "Manufacturer", "Model", "Mileage", "Power_HP", "Fuel",
                "Drive_type"]

df_step11 = rearrange_columns(df_step10_2, first_columns)

In [ ]:
print("")
print("########################## Finished 11. Rearrange Columns ##########################")
print("")
print("")
print("")
print("")

# Data Cleaning Overview

In [ ]:
print("########################## 12. Data Cleaning Overview ##########################")
print("")

Show Cleaned Data

In [ ]:
df_step11

Give Information about cleaning process

In [ ]:
data_cleaning_overview(scraped_data, df_step11)

In [ ]:
print("")
print("########################## Finished 12. Data Cleaning Overview ##########################")
print("")
print("")
print("")
print("")

# Data Saving

## Whole Data

In [ ]:
print("########################## 13. Data Saving ##########################")

In [ ]:
save_result_df = True

output_path = "Data//Cleaned Data"

if save_result_df:
    save_clean_data(df_step11, output_path)

In [ ]:
print("")
print("########################## Finished 13. Data Saving ##########################")

## Helper Data

In [ ]:
save_helper_df = True

output_path_helper = "Data//Helper Data//Cleaned Data//"

if save_helper_df:
    helper_data = df_step11.iloc[0:1]
    save_clean_data(helper_data, output_path_helper)

# Clear Work Space

Automatically Clear Workspace at the end of the script to ensure that no temporary variables are kept when this script is run over the Major Pipeline Terminal

In [ ]:
clear_workspace = True
if clear_workspace:
    %reset -f